# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096900


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.42s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:59,  2.20s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:41,  1.58s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:27,  1.13s/it]

...

Rendering models:  70%|███████   | 21/30 [00:08<00:01,  5.81it/s]

Rendering models:  77%|███████▋  | 23/30 [00:08<00:00,  7.25it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  8.84it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  9.03it/s]

not-logged-in-3b0b929f4475fd276561    0.000350
Alexander00                           0.000638
ellivc                                0.002454
HM01                                  0.148445
jnarayanbvg                           0.028721
Galactea                              0.001822
gracie_solveig                        0.000714
not-logged-in-ed24b424692bed06d715    0.002184
rmz4160                               0.000499
bentleyghioda                         0.000216
not-logged-in-6e654a118e00ea29ed67    0.000876
lsautter                              0.000334
not-logged-in-a92559927bfd05e023e7    0.004080
TheEpicPrimius123                     0.005238
not-logged-in-6d3e4eb12f2149553110    0.000413
Abigail_Louise                        0.001687
arad98                                0.000318
not-logged-in-3ae935e3afaee843f530    0.025291
Lavadude                              0.015102
helios1235                            0.000277
LeusaneLordelo                        0.000613
slambier     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())